# Spain Electricity Shortfall Challenge - Kaggle submission



## Regression Project Student Solution

© Explore Data Science Academy

---

In [1]:
# Libraries for data loading, data manipulation and data visulisation
import numpy as np   
import pandas as pd   

# Libraries for data preparation and model building
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn import *
from sklearn.linear_model import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


from tqdm import tqdm
import math
import random

#Visualization Packages
from scipy import stats
from scipy.stats import norm, skew 
import seaborn as sns 
from matplotlib import pyplot as plt


In [2]:
# Load the test data
df_test = pd.read_csv('Data/df_test.csv')
df_test.tail()

,Unnamed: 0,time,Madrid_wind_speed,Valencia_wind_deg,Bilbao_rain_1h,Valencia_wind_speed,Seville_humidity,Madrid_humidity,Bilbao_clouds_all,Bilbao_wind_speed,...,Barcelona_temp_max,Madrid_temp_max,Barcelona_temp,Bilbao_temp_min,Bilbao_temp,Barcelona_temp_min,Bilbao_temp_max,Seville_temp_min,Madrid_temp,Madrid_temp_min
2915,11678,2018-12-31 09:00:00,0.333333,level_9,0.0,2.000000,81.666667,49.666667,80.000000,1.333333,...,279.816667,281.483333,278.140000,270.816667,273.210000,276.483333,276.150000,279.816667,274.910000,271.150000
2916,11679,2018-12-31 12:00:00,0.333333,level_8,0.0,1.333333,61.000000,28.333333,56.666667,1.000000,...,286.483333,287.816667,286.150000,278.150000,278.443333,285.816667,278.816667,287.150000,283.156667,280.483333
2917,11680,2018-12-31 15:00:00,1.000000,level_6,0.0,3.000000,47.000000,26.333333,0.000000,0.666667,...,289.483333,288.816667,288.820000,284.150000,285.073333,288.150000,285.816667,290.816667,287.733333,286.483333
2918,11681,2018-12-31 18:00:00,1.000000,level_6,0.0,2.000000,52.666667,56.666667,0.000000,0.666667,...,285.816667,285.150000,284.473333,280.150000,281.626667,283.150000,282.816667,287.483333,283.813333,282.150000
2919,11682,2018-12-31 21:00:00,1.333333,level_10,0.0,2.333333,61.666667,69.333333,0.000000,1.333333,...,283.816667,276.816667,281.133333,276.150000,276.450000,278.483333,276.816667,283.816667,276.623333,276.483333


In [ ]:
predicted_y = pd.DataFrame(predictions_Rfr_test, columns = ['load_shortfall_3h'])
predicted_y.head()

In [ ]:
predicted_y.describe().round()

In [ ]:
y.describe().round()

In [ ]:
output = pd.DataFrame({'time':df_test['time']})
team_submission_Lr = output.join(submitted_y)
team_submission_Lr.to_csv('team_NM2_submission.csv', index = False)